# Using Pre-Trained Models Project

In this project we will import a pre-existing model that recognizes objects and use the model to identify those objects in a video. We'll edit the video to draw boxes around the identified object and then reassemble the video so that the boxes are shown around objects in the video.

## Team

*   *Prashamsa Rimal*
*   *Christine Zhu*
*   *Rosemary Austin*



# **1. Obtain a video file from Pixabay to use for classification.**

In [0]:
#This is loading in the video file

import cv2 as cv
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd

cars_video = cv.VideoCapture('cars.mp4')

height = int(cars_video.get(cv.CAP_PROP_FRAME_HEIGHT))
width = int(cars_video.get(cv.CAP_PROP_FRAME_WIDTH))
fps = cars_video.get(cv.CAP_PROP_FPS)
total_frames = int(cars_video.get(cv.CAP_PROP_FRAME_COUNT))

cars_video.release()

# **2. Obtain the pre-trained model from the TensorFlow Zoo.**

In [0]:
#This is uploading the model 

import urllib.request
import os

base_url = 'http://download.tensorflow.org/models/object_detection/'
file_name = 'ssd_mobilenet_v1_coco_2018_01_28.tar.gz'

url = base_url + file_name

urllib.request.urlretrieve(url, file_name)

os.listdir()

In [0]:
# Extract the Model Data
import tarfile
import shutil

dir_name = file_name[0:-len('.tar.gz')]
if os.path.exists(dir_name):
  shutil.rmtree(dir_name) 

tarfile.open(file_name, 'r:gz').extractall('./')

os.listdir(dir_name)

# **3. Load the pre-trained model into a TensorFlow object.**

In [0]:
#Loading the graph
frozen_graph = os.path.join(dir_name, 'frozen_inference_graph.pb')

with tf.gfile.FastGFile(frozen_graph,'rb') as f:
  graph_def = tf.GraphDef()
  graph_def.ParseFromString(f.read())

In [0]:
outputs = (
    'num_detections',
    'detection_classes',
    'detection_scores',
    'detection_boxes',
)

In [0]:
#This resizes it in readable form
def frame_fix(image):
  frame = cv.resize(image, (300, 300))
  return frame


In [0]:
#import the dataset library
open_new = open("labels.txt", "r")
labels={}
for x in open_new:
  key,_,value=x.partition(':')
  value,_,_=value.partition('\n')
  labels[key]=value

In [0]:
labels

In [0]:
#importing the loading bar
from tqdm import tqdm_notebook as tqdm

In [0]:
cars_video = cv.VideoCapture('cars.mp4')

# **4. Process the video frame-by-frame creating a modified output video.**
# **5. Apply a classification model to an image.**
# **6. Draw a bounding box around classified objects in each image.**

In [0]:
# 4. Process the car video frame-by-frame to create a modified out video

total_frames = int(cars_video.get(cv.CAP_PROP_FRAME_COUNT))
fourcc = cv.VideoWriter_fourcc(*'mp4v')
output_video = cv.VideoWriter('Cars-short.mp4', fourcc, fps, (300,300))

#looping through each frame
for current_frame in tqdm(range(0, total_frames,30)): #Capture a frame for every 50 frames
  cars_video.set(cv.CAP_PROP_POS_FRAMES, current_frame)
  ret, frame = cars_video.read()  
  frame=cv.resize(frame,(300,300)) #resize the frame to 300*300
  if not ret:
    raise Exception(f'Problem reading frame {current_frame} from video')
  image = [frame] #image into a list 
  
  #5. starts tensor flow sess to create the graph 
  with tf.Session() as sess:
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')

    detections = sess.run(
        [sess.graph.get_tensor_by_name(f'{op}:0') for op in outputs],
        feed_dict={ 'image_tensor:0': image })
    
    # 6. Draw a bounding box around classified objects in each image  
    height,width,colour = frame.shape
    for det in range(int(detections[0][0])):
      left = detections[3][0][det][1] *width
      top = detections[3][0][det][0] * height
      right = detections[3][0][det][3] *width
      bottom = detections[3][0][det][2] *height
      frame = cv.rectangle(frame,(int(left),int(top)),(int(right),int(bottom)),[0,255,0],2)
      label = labels[str(int(detections[1][0][det]))]
      frame = cv.putText(frame, label , (int(left),int(top)), cv.FONT_HERSHEY_SIMPLEX,1,[0,255,0],2)
    
    #create the video   
    output_video.write(frame)
    
#release the video
output_video.release()